In [113]:
import matplotlib.pyplot as plt
import plotly.express as px
from tqdm import tqdm
import pandas as pd
import time
import csv
import os

## Funzione per trasformare i dati in CSV
Lo script prende in input un file .txt con il parametro 'rb' quindi in lettura in formato bytecode che vuol dire che dovrà essere decodificato.
Per prima cosa si scorre fino alla riga che inizia con 'NR' perché i dati iniziano da li, quando la trovo imposto flag_data a 1 che serve a dirmi che da li in poi dovrò salvare le righe.
Cerco la prima riga che inizia con '0' e la salto settando firsr_row a 1.
Quando sia data_flag che first_row sono a 1 allora inizio a salvare le righe.

Per salvare le righe:
* prima le decodifico con .decode()
* poi le divido con .split() che se usato senza parametri splitta la stringa in base agli spazi
* unisco i valori delle ultime due colonne perché fanno entrambi riferimento alla data
* elimino l'ultima riga perché l'ho unita alla penultima
* faccio un append della riga


In [124]:
def txt_to_csv(filename, filedest):
    
    
    if os.path.isfile(filedest + '.csv'):
        os.remove(filedest + '.csv')
        
    file1 = open(filename + '.txt', "rb")
    file_csv = open(filedest + '.csv', 'w+', newline ='')

    count = 0
    flag_data = 0

    with file_csv:
        for line in file1:

            if b'\xb0C' in line:     # se trovo il carattere °C lo ignoro
                pass
            else:
                line = line.decode() # altrimenti decodifico la riga

                # cerco la riga con i nomi della tabella
                if line.startswith('NR'):
                    flag_data = 1                  # quando trovo i nomi delle colonne imposto flag_data = 1
                    table_names = line.split()     # divido i nomi delle colonne      
                    table_names.remove('LastLine') # elimino 'LastLine' che fa riferimento ad un singolo dato duplicato
                    #for item in table_names: print(item)
                    write = csv.writer(file_csv)  
                    write.writerow(table_names)

                # Se sono arrivato alla tabella ma la riga inizia con 0 la salto
                elif flag_data == 1 and line.startswith('0'):
                    pass

                # Se sono arrivato alla tabella (flag_data == 1) e non sono alla prima riga
                # inizio a salvare i dati
                elif flag_data == 1:
                    row = line.split()

                    if not line.startswith('[END]') and len(row) == len(table_names)+1: # le righe con meno di 25 elementi hanno dati mancanti
                        row[-2] = str(row[-2]) + ' ' + str(row[-1]) # unisco l'ora col giorno
                        del row[-1] # elimino la colonna con i giorni
                        write = csv.writer(file_csv)
                        write.writerow(row)
                        count = count+1      # aggiorno il contatore delle righe

    #print(str(count) + ' righe salvate.')

#txt_to_csv('Tabelle_omv/TabLogAntTHS')

Per convertire tutti i file da txt a csv mi basta iterare all'interno della cartella e passare ogni file alla funzione txt_to_csv che prende in input il nome del file da convertire e il percorso di destinazione del file.

In [44]:
import os

for filename in os.listdir('Tabelle_omv/'):
    if filename.endswith(".txt"): 
        print(filename[:-4])
        txt_to_csv(filename = 'Tabelle_omv/' + filename[:-4],
                   filedest = 'Tabelle_omv/csv/' + filename[:-4])
        print('\n')

TabLogPostTHS3
file esistente
14399 righe salvate.


TabLogPostTHS4
file esistente
14399 righe salvate.


DiagnosisTabl2
file esistente
14398 righe salvate.


TabLogFMS2_SX
file esistente
14399 righe salvate.


TabLogFMS3_SX
file esistente
14399 righe salvate.


TabLogAntTHS4
file esistente
14399 righe salvate.


DiagnosisTabl3
file esistente
14398 righe salvate.


DiagnosisTable
file esistente
14389 righe salvate.


TabLogFMS3_DX
file esistente
14399 righe salvate.


DiagnosisTable4
file esistente
14390 righe salvate.


TabLogPostTHS
file esistente
14399 righe salvate.


TabLogFMS2_DX
file esistente
14399 righe salvate.


DiagnosisTabl4
file esistente
14350 righe salvate.


TabLogFMS4_SX
file esistente
14399 righe salvate.


TabLogAntTHS
file esistente
14399 righe salvate.


TabLogAntTHS2
file esistente
14399 righe salvate.


DiagnosisTable3
file esistente
14390 righe salvate.


TabLogPostTHS2
file esistente
14399 righe salvate.


TabLogFMS_DX
file esistente
14399 righe salvate.


Tab

## Funzione per caricare il CSV in un DataFrame
La funzione prende in input il nome del file .csv da caricare, converte la colonna DataTime in datetime e la imposta come indice del dataframe. Controlla se il dataframe è ordinato e se non lo è lo riordina. Elimina la colonna NR. Restituisce il dataframe.

In [7]:
def load_data(file):
    #column_subset = ['t_Mach',   't_HydrTa', 't_TTlubr', 't_TT_ab1', 
                     #'t_TT_ab2', 't_HdStUp', 't_HdStLw', 't_Sfrnt1', 
                     #'t_Sfrnt2', 't_Srear',  't_Smotor', 'DataTime']

    df = pd.read_csv (file,
                      #usecols=column_subset,
                      index_col=False)

    df['DataTime'] = pd.to_datetime(df['DataTime'], format='%H:%M:%S %d.%m.%Y')
    
    df.set_index('DataTime', drop = True, inplace=True)
    
    if not df.index.is_monotonic:
        df.sort_index(inplace = True)
    
    df.drop('NR', axis=1, inplace=True)
    
    return df

In [14]:
import os

for filename in os.listdir('Tabelle_omv/csv/'):
    if filename.endswith(".csv"): 
        print(filename)
        data = load_data('Tabelle_omv/csv/' + filename)
        data.info()
        print('\n')

TabLogFMS3_DX.csv
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 14399 entries, 2020-03-02 21:00:01 to 2020-03-13 03:24:01
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   t_Mach    14399 non-null  float64
 1   t_HydrTa  14399 non-null  float64
 2   t_TTlubr  14399 non-null  float64
 3   t_TT_ab1  14399 non-null  float64
 4   t_TT_ab2  14399 non-null  float64
 5   t_HdStUp  14399 non-null  float64
 6   t_HdStLw  14399 non-null  float64
 7   t_Sfrnt1  14399 non-null  float64
 8   t_Sfrnt2  14399 non-null  float64
 9   t_Srear   14399 non-null  float64
 10  t_Smotor  14399 non-null  float64
 11  t_ReqDom  14399 non-null  float64
 12  t_ExtDom  14399 non-null  int64  
 13  t_ProDom  14399 non-null  int64  
 14  t_PalFMS  14399 non-null  float64
dtypes: float64(13), int64(2)
memory usage: 1.8 MB


TabLogFMS_SX.csv
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 14399 entries, 2020-03-23 03:37:01 to 2020-04-02 04:


TabLogFMS_DX.csv
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 14399 entries, 2020-03-23 03:38:01 to 2020-04-02 04:45:01
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   t_Mach    14399 non-null  float64
 1   t_HydrTa  14399 non-null  float64
 2   t_TTlubr  14399 non-null  float64
 3   t_TT_ab1  14399 non-null  float64
 4   t_TT_ab2  14399 non-null  float64
 5   t_HdStUp  14399 non-null  float64
 6   t_HdStLw  14399 non-null  float64
 7   t_Sfrnt1  14399 non-null  float64
 8   t_Sfrnt2  14399 non-null  float64
 9   t_Srear   14399 non-null  float64
 10  t_Smotor  14399 non-null  float64
 11  t_ReqDom  14399 non-null  int64  
 12  t_ExtDom  14399 non-null  int64  
 13  t_ProDom  14399 non-null  int64  
 14  t_PalFMS  14399 non-null  float64
dtypes: float64(12), int64(3)
memory usage: 1.8 MB


TabLogFMS4_DX.csv
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 14399 entries, 2020-02-21 18:43:01 to 2020-03-02 20

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 14399 entries, 2020-02-21 22:50:01 to 2020-03-03 01:05:01
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   t_Mach    14399 non-null  float64
 1   t_HydrTa  14399 non-null  float64
 2   t_TTlubr  14399 non-null  float64
 3   t_TT_ab1  14399 non-null  float64
 4   t_TT_ab2  14399 non-null  float64
 5   t_HdStUp  14399 non-null  float64
 6   t_HdStLw  14399 non-null  float64
 7   t_Sfrnt1  14399 non-null  float64
 8   t_Sfrnt2  14399 non-null  float64
 9   t_Srear   14399 non-null  float64
 10  t_Smotor  14399 non-null  float64
 11  t_ReqDom  14399 non-null  int64  
 12  t_ExtDom  14399 non-null  int64  
 13  t_ProDom  14399 non-null  int64  
 14  t_Envir   14399 non-null  float64
dtypes: float64(12), int64(3)
memory usage: 1.8 MB


TabLogFMS2_DX.csv
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 14399 entries, 2020-03-13 03:26:01 to 2020-03-23 03:36:01
Data column

In [11]:
path = 'Tabelle_omv/csv/'

data1 = load_data(path + 'TabLogAntTHS.csv')
data2 = load_data(path + 'TabLogAntTHS2.csv')
data3 = load_data(path + 'TabLogAntTHS3.csv')
data4 = load_data(path + 'TabLogAntTHS4.csv')

data_all = pd.concat([data1, data2, data3, data4])
print(data_all.index.is_monotonic)
data_all.sort_index(inplace = True)
print(data_all.index.is_monotonic)

print(pd.date_range(start = '2019-06-15 19:27:01', end = '2020-03-30 12:33:01' ).difference(data_all.index))

False
True
DatetimeIndex(['2019-06-15 19:27:01', '2019-06-16 19:27:01',
               '2019-06-17 19:27:01', '2019-06-18 19:27:01',
               '2019-06-19 19:27:01', '2019-06-20 19:27:01',
               '2019-06-21 19:27:01', '2019-06-22 19:27:01',
               '2019-06-23 19:27:01', '2019-06-24 19:27:01',
               ...
               '2020-02-14 19:27:01', '2020-02-15 19:27:01',
               '2020-02-16 19:27:01', '2020-02-17 19:27:01',
               '2020-02-18 19:27:01', '2020-02-19 19:27:01',
               '2020-02-20 19:27:01', '2020-02-21 19:27:01',
               '2020-02-22 19:27:01', '2020-02-23 19:27:01'],
              dtype='datetime64[ns]', length=254, freq=None)


## Processo per caricare tutti i file dello stesso tipo

In [134]:
path = 'Tabelle_omv/'
data_file = 'DiagnosisTable'
data_list=[]

for filename in os.listdir(path):
    if filename.endswith(".txt") and data_file in filename: 
        print(filename[:-4])
        txt_to_csv(filename = path + filename[:-4],
                   filedest = path + 'csv/' + filename[:-4])
        
        data = load_data(path + 'csv/' + filename[:-4] + '.csv')
        data_list.append(data)
        
data_all = pd.concat(data_list)
data_all.sort_index(inplace = True)


data_all.info()
print(pd.date_range(start = '2020-02-21 22:50:01', end = '2020-04-02 08:52:01' ).difference(data_all.index))
#data_all = data_all.loc['2020-04-03 01:05:00':'2020-05-03 23:05:00']
data_all.drop('OpMode', axis=1, inplace=True)
data_all = data_all.resample('1D').mean()
#data_all = data_all.loc['2020-04-01 23:05:00':'2020-05-05 23:05:00']
#data_all['t_Mach'].plot(figsize=(15,10))
#data_all.info()

#for index, row in data_all.iterrows():
#    print(index, row['t_Mach'])


fig = px.line(data_all)
fig.show()

DiagnosisTable
DiagnosisTable4
DiagnosisTable3
DiagnosisTable2
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 57559 entries, 2019-06-15 19:27:01 to 2020-03-30 12:33:01
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   t_Mach    57559 non-null  float64
 1   t_HydrTa  57559 non-null  float64
 2   t_TTlubr  57559 non-null  float64
 3   t_TT_ab1  57559 non-null  float64
 4   t_TT_ab2  57559 non-null  float64
 5   t_HdStUp  57559 non-null  float64
 6   t_HdStLw  57559 non-null  float64
 7   t_Sfrnt1  57559 non-null  float64
 8   t_Sfrnt2  57559 non-null  float64
 9   t_Srear   57559 non-null  float64
 10  t_Smotor  57559 non-null  float64
 11  OpMode    57559 non-null  object 
 12  S_Speed   57559 non-null  int64  
 13  S_Load    57559 non-null  int64  
 14  tCo_H_X1  57559 non-null  int64  
 15  tCo_H_Y1  57559 non-null  float64
 16  tCo_H_Z1  57559 non-null  float64
 17  tCo_H_X2  57559 non-null  int64  
 18  tCo_H_Y2

In [ ]:
df = pd.read_csv ('parsed_data.csv', index_col=False)

for index, row in df.iterrows():
    if row['tCo_S2_Z'] != '+0':
        print(str(index) + ' ' + str(row['NR']) + ' ' + str(row['tCo_S2_Z']))
        df.drop(index, inplace=True)

fig, axs = plt.subplots(18,figsize=(15,50))
fig.suptitle('Vertically stacked subplots')
axs[0].plot(df['t_Mach'])
axs[0].set_title('t_Mach')

axs[1].plot(df['t_HydrTa'])
axs[1].set_title('t_Mach')

axs[2].plot(df[['t_TTlubr','t_TT_ab1','t_TT_ab2']])
axs[2].set_title('t_TTlubr, t_TT_ab1, t_TT_ab2')

axs[3].plot(df['t_HdStUp'])
axs[3].set_title('t_HdStUp')

axs[4].plot(df['t_HdStLw'])
axs[4].set_title('t_HdStLw')

axs[5].plot(df[['t_Sfrnt1', 't_Sfrnt2']])
axs[5].set_title('t_Sfrnt1, t_Sfrnt2')

axs[6].plot(df['t_Srear'])
axs[6].set_title('t_Srear')

axs[7].plot(df['t_Smotor'])
axs[7].set_title('t_Smotor')

axs[8].plot(df['S_Speed'])
axs[8].set_title('S_Speed')

axs[9].plot(df['S_Load'])
axs[9].set_title('S_Load')

axs[10].plot(df['tCo_H_X1'])
axs[10].set_title('tCo_H_X1')

axs[11].plot(df['tCo_H_Y1'])
axs[11].set_title('tCo_H_Y1')

axs[12].plot(df['tCo_H_Z1'])
axs[12].set_title('tCo_H_Z1')

axs[13].plot(df['tCo_H_X2'])
axs[13].set_title('tCo_H_X2')

axs[14].plot(df['tCo_H_Y2'])
axs[14].set_title('tCo_H_Y2')

axs[15].plot(df['tCo_H_Z2'])
axs[15].set_title('tCo_H_Z2')

axs[16].plot(df['tCo_H_Y3'])
axs[16].set_title('tCo_H_Y3')

axs[17].plot(df['tCo_S2_Z'])
axs[17].set_title('tCo_S2_Z')
